# KVLCC2 in motions

# Purpose
This notebook analyzes the first results from roll decay simulations in Motions (without vicsous damping).

# Methodology
Quickly describe assumptions and processing steps.

# Setup

In [ ]:
# %load imports.py
"""
These is the standard setup for the notebooks.
"""

%matplotlib inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import OrderedDict
#plt.style.use('paper')

#import data
import copy
from mdldb.run import Run

from sklearn.pipeline import Pipeline
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.direct_estimator_cubic import EstimatorQuadraticB, EstimatorCubic
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator
import rolldecayestimators.equations as equations
import rolldecayestimators.lambdas as lambdas
from rolldecayestimators.substitute_dynamic_symbols import lambdify
import rolldecayestimators.symbols as symbols
import sympy as sp

from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex

from sklearn.metrics import r2_score
from src.data import database
from mdldb import tables
import shipflowmotionshelpers.shipflowmotionshelpers as helpers
import shipflowmotionshelpers.preprocess as preprocess


In [ ]:
import joblib
from src.helpers import get_ikeda, calculate_ikeda, get_estimator_variation, get_data_variation 

## Load data from Motions:

In [ ]:
file_paths = [
    '../data/external/kvlcc2_rolldecay_0kn',
]
#file_path_viscous = 'data/Final_use_in_parametric_roll_simulations/m5030_HADR_B100_Q100_rolldecay_10kn_TS.csv'

#df = helpers.load_time_series(file_path_no_viscous)  # No vicous damping
#df_visc = helpers.load_time_series(file_path_viscous) # Including vicous

In [ ]:
df_parameters = pd.DataFrame()
df_parameters =  helpers.load_parameters(file_path=file_paths)
#df_parameters.rename(index={'kvlcc2_rolldecay_0kn':'inviscid'}, inplace=True)
df_parameters

In [ ]:
time_series = helpers.load_time_series(df_parameters=df_parameters)

In [ ]:
df = time_series['kvlcc2_rolldecay_0kn']
X = df
fig,ax=plt.subplots()
X.plot(y='phi', ax=ax)

## Load MDL results

In [ ]:
db = database.get_db()

sql = """
SELECT * from run
INNER JOIN loading_conditions
ON (run.loading_condition_id = loading_conditions.id)
INNER JOIN models
ON (run.model_number = models.model_number)
INNER JOIN ships
ON (run.ship_name = ships.name)
WHERE run.model_number='M5057-01-A' and run.test_type='roll decay' and run.project_number=40178362;
"""
df_rolldecays = pd.read_sql(sql=sql, con=db.engine)
df_rolldecays['rho']=1000
df_rolldecays['g']=9.81
df_rolldecays=df_rolldecays.loc[:,~df_rolldecays.columns.duplicated()]
df_rolldecays.set_index('id', inplace=True)

df_rolldecays['ship_speed'].fillna(0, inplace=True)
df_rolldecays['R'] = 2.4 # Bilge keel radius measured on geometry (full scale)

df_rolldecays=df_rolldecays.loc[[21338,21340,]].copy()

In [ ]:
run_paths={
    21338 : {
        'scores_indata_path':'../models/KVLCC2_speed.IN',
        'scores_outdata_path':'../data/interim/KVLCC2_speed.out',
        'roll_decay_model':'../models/KVLCC2_0_speed.pkl',
        'motions_file_paths': ['kvlcc2_rolldecay_0kn'],
            
            },
    21340 : {
        'scores_indata_path':'../models/KVLCC2_speed.IN',
        'scores_outdata_path':'../data/interim/KVLCC2_speed.out',
        'roll_decay_model':'../models/KVLCC2_speed.pkl',
    }
}

In [ ]:
runs = OrderedDict()

for run_id, run in run_paths.items():
    
    mdl_meta_data = df_rolldecays.loc[run_id]
    runs[run_id] = new_run = {
        'motions':{},
    }
    
    ## MDL:
    model_mdl = joblib.load(run['roll_decay_model'])
    estimator_mdl = model_mdl['estimator']
    estimator_mdl.calculate_amplitudes_and_damping()
    new_run['model_mdl']=model_mdl
    new_run['estimator_mdl']=estimator_mdl
    
    scale_factor = mdl_meta_data.scale_factor
    new_run['meta_data'] = meta_data={
            'Volume':mdl_meta_data.Volume/(scale_factor**3),
            'GM':mdl_meta_data.gm/scale_factor,
            'rho':mdl_meta_data.rho,
            'g':mdl_meta_data.g,
            'beam':mdl_meta_data.beam/scale_factor,
        }
    
    new_run['results'] = results = estimator_mdl.result_for_database(meta_data=meta_data)
    
    # Prediction
    new_run['df_model'] = get_estimator_variation(estimator = estimator_mdl, results=results, meta_data=meta_data)
    
    # Model tests
    new_run['df'] = get_data_variation(estimator = estimator_mdl, results=results, meta_data=meta_data)
    phi_a = new_run['df']['phi_a']
    
    ## Motions
    for motions_file_path in run.get('motions_file_paths',[]):
        motion_file = new_run['motions'][motions_file_path] = {}
        
        motion_file['parameters'] = parameters = df_parameters.loc[motions_file_path]
        
        motion_file['X'] = X = time_series[motions_file_path]
        
                
        motion_file['model'] = model = EstimatorCubic(p0=estimator_mdl.parameters)
        model.fit(X=X)
        assert model.score() > 0.99
        
        motion_file['meta_data'] = meta_data ={
            'Volume':parameters.V,
            'GM':mdl_meta_data.gm/mdl_meta_data.scale_factor,
            'rho':parameters.dens,
            'g':parameters.gravi,
            'beam':parameters.B,
        }
    
        results = model.result_for_database(meta_data=meta_data)
        motion_file['results'] = results
        model.calculate_amplitudes_and_damping()
        
        # Prediction
        motion_file['df_model'] = get_estimator_variation(estimator = model, results = results, meta_data=meta_data)
                
        # Simulation
        motion_file['df'] = get_data_variation(estimator = model, results = results, meta_data=meta_data)
        
                
    ## Ikeda
    new_run['ikeda'] = ikeda =  {}
    omega0=new_run['results']['omega0']     
    ikeda['estimator'] = ikeda_estimator = get_ikeda(indata_file_path=run['scores_indata_path'], output_file_path=run['scores_outdata_path'], mdl_meta_data=mdl_meta_data, omega0=omega0, phi_a=phi_a)
    ikeda['df'] = results = calculate_ikeda(ikeda = ikeda_estimator)
    results['phi_a'] = phi_a
    results.set_index('phi_a', inplace=True)
    


In [ ]:
run_id=21338
run = runs[run_id]

<a id='damping'></a>

In [ ]:
df_ikeda = run['ikeda']['df']

fig,ax=plt.subplots()
interesting_ = ['B_L_hat','B_W_hat','B_F_hat','B_E_hat',]
df_ikeda.plot.area(y=interesting_, ax=ax)

## Model test
run['df'].plot(x='phi_a', y='B_hat', style='b.', label='Model test', ax=ax)
run['df_model'].plot(x='phi_a', y='B_e_hat', style='-', label='Cubic model test', ax=ax)

## Motions
motion_file = run['motions']['kvlcc2_rolldecay_0kn']
motion_file['df'].plot(x='phi_a', y='B_hat', style='k.', label='Motions inviscid', ax=ax)
motion_file['df_model'].plot(x='phi_a', y='B_e_hat', style='-', label='Cubic model Motions', ax=ax)
ax.legend()

ax.set_ylim(0,0.003)